Download labeling from GitHub - https://github.com/tzutalin/labelImg


`!pip install pyqt5`

`!pip install lxml`


Installation guide - https://github.com/heartexlabs/labelImg#installation

Run this command in the terminal

`pyrcc5 -o libs/resources.py resources.qrc`

In [3]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.retinanet import RetinaNet_ResNet50_FPN_Weights

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Number of classes including background
num_classes = 91 

# Load a pre-trained model
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True, num_classes=num_classes)

model.to(device)

# Continue with your code...(Origional numbers)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)


class YOLODataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform if transform else transforms.ToTensor()
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))
        self.labels = list(sorted(os.listdir(os.path.join(root_dir, "labels"))))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])
        label_path = os.path.join(self.root_dir, "labels", self.labels[idx])

        img = Image.open(img_path).convert("RGB")
        img_width, img_height = img.size

        # Apply transformation after getting original size
        img = self.transform(img)

        # Read YOLO label file
        with open(label_path, "r") as file:
            lines = file.read().splitlines()

        boxes = []
        labels = []
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.split())
            labels.append(int(class_id))

            x_min = img_width * (x_center - width / 2)
            y_min = img_height * (y_center - height / 2)
            x_max = img_width * (x_center + width / 2)
            y_max = img_height * (y_center + height / 2)
            boxes.append([x_min, y_min, x_max, y_max])

        target = {}
        target['boxes'] = torch.tensor(boxes, dtype=torch.float32)
        target['labels'] = torch.tensor(labels, dtype=torch.int64)

        return img, target

# Define your own paths here
train_dataset = YOLODataset("data/train set")
valid_dataset = YOLODataset("data/validation set")

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]

    return images, targets

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)


In [4]:
model.train()
num_epochs = 500
model = model.to(device)
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_dataloader):
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]


        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch: {epoch + 1}, Loss: {losses.item()}")

Epoch: 1, Loss: 0.43990445137023926
Epoch: 2, Loss: 0.38618481159210205
Epoch: 3, Loss: 0.20292797684669495
Epoch: 4, Loss: 0.20639599859714508
Epoch: 5, Loss: 0.16347703337669373
Epoch: 6, Loss: 0.1724870204925537
Epoch: 7, Loss: 0.12876035273075104
Epoch: 8, Loss: 0.098701611161232
Epoch: 9, Loss: 0.20843347907066345
Epoch: 10, Loss: 0.1403430700302124
Epoch: 11, Loss: 0.07947002351284027
Epoch: 12, Loss: 0.09271256625652313
Epoch: 13, Loss: 0.12376318871974945
Epoch: 14, Loss: 0.09589266031980515
Epoch: 15, Loss: 0.13618016242980957
Epoch: 16, Loss: 0.08114887028932571
Epoch: 17, Loss: 0.1088576465845108
Epoch: 18, Loss: 0.10441714525222778
Epoch: 19, Loss: 0.09452976286411285
Epoch: 20, Loss: 0.1113516166806221
Epoch: 21, Loss: 0.10474219918251038
Epoch: 22, Loss: 0.07956115156412125
Epoch: 23, Loss: 0.10429853945970535
Epoch: 24, Loss: 0.06037108600139618
Epoch: 25, Loss: 0.08999770879745483
Epoch: 26, Loss: 0.0896042063832283
Epoch: 27, Loss: 0.10325322300195694
Epoch: 28, Loss: 